In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv("netflix_titles.csv")


df = df.dropna(subset=["type", "listed_in", "country", "duration"])

df["recommended"] = df["listed_in"].apply(
    lambda x: 1 if ("Drama" in x or "Thriller" in x) else 0
)

X = df[["type", "country", "release_year", "duration", "listed_in"]]
y = df["recommended"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

n_models = 10
n_samples = len(X_train_scaled)
models = []
probabilities = []

for i in range(n_models):
    # Bootstrap sampling
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    Xb = X_train_scaled[indices]
    yb = y_train.iloc[indices]

    # Train Decision Tree
    model = RandomForestClassifier(n_estimators=20, max_depth=200, random_state=42)
    model.fit(Xb, yb)
    models.append(model)

for model in models:
    y_proba = model.predict_proba(X_test_scaled)
    probabilities.append(y_proba)

avg_proba = np.mean(probabilities, axis=0)
ensemble_pred = np.argmax(avg_proba, axis=1)

# Accuracy
acc = accuracy_score(y_test, ensemble_pred)
print(f"🎯 Netflix Recommendation Classifier Accuracy: {acc:.4f}")

def recommend_show(genre, country, year):
    sample = pd.DataFrame({
        "type": ["Movie"],
        "country": [country],
        "release_year": [year],
        "duration": ["90 min"],
        "listed_in": [genre]
    })

🎯 Netflix Recommendation Classifier Accuracy: 0.9712
